#### 1. Get the data into a Pandas DataFrame.


In [1]:
import csv
import requests
import pandas as pd
import openpyxl

link = "https://docs.google.com/spreadsheets/d/1isOVWu-C6UqGy7Kg7sabE6O_Uq1AsMgK/edit#gid=215309859"

# Extract the gid from the URL
gid = link.split("#gid=")[1]

# Use the gid to download the sheet as a CSV file
link = f"https://docs.google.com/spreadsheets/d/1isOVWu-C6UqGy7Kg7sabE6O_Uq1AsMgK/export?gid={gid}&format=csv"
response = requests.get(link)


# Check if the request was successful
if response.status_code == 200:
    # If the request was successful, the data is stored in the response content
    data = response.content
else:
    # If the request was not successful, print an error message
    print("Failed to fetch data")

# Parse the data and extract the year and price for each car
reader = csv.reader(data.decode("utf-8").splitlines())

# Convert the reader object to a list
csv_data = list(reader)

# Get the column names from the first row of the CSV file
column_names = csv_data[0]

# Remove the first row from the data
data = csv_data[1:]

# Create the DataFrame, specifying the column names
df = pd.DataFrame(data, columns=column_names)

df['Price'] = pd.to_numeric(df['Price'])
df['Year'] = pd.to_numeric(df['Year'])
df['Km'] = pd.to_numeric(df['Km'])
df['Power'] = pd.to_numeric(df['Power'])
df['Engine'] = pd.to_numeric(df['Engine'])


## 2. Data Cleaning

Remove the cars where there exists the same car with the same values for all the columns except the price.


In [2]:
# Create a list of columns to keep
columns_to_keep = df.columns.difference(['Price']).to_list()

# Create a copy of the DataFrame
df_sorted = df.copy()

# Sort the copy by price
df_sorted.sort_values('Price', ascending=True, inplace=True)

# Drop duplicates
df_sorted = df_sorted.drop_duplicates(subset=columns_to_keep, keep='first')

# Filter the original DataFrame
df = df[df.isin(df_sorted)].dropna(how='all')

# Save the DataFrame to an Excel file
df.to_excel("Car_Sales_Data_Analysis.xlsx", index=False)


#### 3. Find the average price of cars for each years and return it as a dictionary.


In [3]:
# Group the data by the "Year" column and compute the average price for each group
price_by_year = df.groupby('Year')['Price'].mean()

# Convert the index to integers
price_by_year.index = price_by_year.index.astype(int)

# Convert the Series to a dictionary
price_by_year_dict = price_by_year.to_dict()

print(price_by_year_dict)


{1970: 42500.0, 1971: 5000.0, 1972: 40000.0, 1974: 28879.8, 1975: 60833.333333333336, 1976: 48155.0, 1977: 59900.0, 1978: 15749.75, 1979: 10316.666666666666, 1980: 23863.333333333332, 1981: 17250.0, 1982: 9449.5, 1983: 15230.0, 1984: 14968.625, 1985: 27872.727272727272, 1986: 59858.1875, 1987: 23983.823529411766, 1988: 15481.25, 1989: 25854.545454545456, 1990: 30047.551020408162, 1991: 23603.048387096773, 1992: 17856.15254237288, 1993: 15343.923076923076, 1994: 22649.296296296296, 1995: 28830.256756756757, 1996: 42260.24074074074, 1997: 37887.72142857143, 1998: 50030.78857142857, 1999: 25564.219178082192, 2000: 33345.17307692308, 2001: 32968.1027027027, 2002: 47636.53333333333, 2003: 51246.65497076023, 2004: 43226.37464788732, 2005: 47998.89935064935, 2006: 61742.11023622047, 2007: 82945.01351351352, 2008: 90628.55584415584, 2009: 93967.71955719557, 2010: 108081.57320099256, 2011: 126674.67919075144, 2012: 160919.57066841415, 2013: 179572.6089527027, 2014: 168873.47517730496, 2015: 197

#### 4. Find the cars with the missing prices and replace them with the average prices of the years they belong to.


In [4]:
# Find the rows where the 'Price' column is null
null_rows = df[df['Price'].isnull()]

# Iterate through the rows and fill in the null values with the average price for the corresponding year
for i, row in null_rows.iterrows():
    year = row['Year']
    avg_price = price_by_year_dict[year]
    df.at[i, 'Price'] = avg_price


#### 5. Find the standard deviation of the prices of each brand and return it as a dictionary.


In [5]:
# Group the data by the 'Brand' column and compute the standard deviation of the prices for each group
# price_std_by_brand = df.groupby('Brand')['Price'].std() # This is the easy way to do it

# Calculate standard deviation manually
price_std_by_brand = df.groupby('Brand')['Price'].apply(lambda group: (sum((x - group.mean())**2 for x in group) / (len(group) - 1))**0.5)

# Convert the Series to a dictionary
price_std_by_brand_dict = price_std_by_brand.to_dict()

print(price_std_by_brand_dict)


{'Alfa Romeo': 37002.304474248216, 'Audi': 175564.01186993002, 'BMW': 136870.8922399194, 'Bentley': 609241.5690739418, 'Chevrolet': 165712.42098807337, 'Chrysler': 40920.14406538657, 'Citroën': 18244.531875917735, 'Dacia': 16941.266278685358, 'Ferrari': 865219.6242592582, 'Fiat': 20426.701650528237, 'Ford': 45378.430715472416, 'Honda': 24652.803100155805, 'Hyundai': 18594.47161147863, 'Jaguar': 179634.94730611893, 'Kia': 22987.55343759756, 'Lada': 3689.326035603285, 'Lancia': 49732.50185815538, 'Maserati': 237530.9574514164, 'Mazda': 33849.29345737826, 'Mercedes - Benz': 477143.5917487099, 'Mini': 37933.199202376665, 'Nissan': 242100.75513615427, 'Opel': 21684.665649875606, 'Peugeot': 19735.4714258816, 'Porsche': 521514.6922331104, 'Renault': 24779.972606418167, 'Rover': 7451.345227249384, 'Seat': 26851.576996933636, 'Skoda': 39746.11714173413, 'Smart': 10548.696601950403, 'Subaru': 41070.009482826375, 'Suzuki': 17343.853440815008, 'Tata': 4726.45649780478, 'Tofaş': 13955.867262677224,

#### 5. Find the average price of each city and return it as a dictionary.


In [6]:
# Group the data by the 'City' column and compute the average price for each group
price_avg_by_city = df.groupby('City')['Price'].mean()

# Convert the Series to a dictionary
price_avg_by_city_dict = price_avg_by_city.to_dict()

print(price_avg_by_city_dict)


{'Adana': 68493.9263803681, 'Adıyaman': 72804.76190476191, 'Afyonkarahisar': 61468.0, 'Aksaray': 59964.25, 'Amasya': 49477.0, 'Ankara': 87488.58665885111, 'Antalya': 109238.6705882353, 'Ardahan': 47875.0, 'Artvin': 34460.0, 'Aydın': 59367.434782608696, 'Ağrı': 39166.5, 'Balıkesir': 54883.39361702128, 'Bartın': 74183.33333333333, 'Batman': 72102.94117647059, 'Bayburt': 65821.25, 'Bilecik': 73911.11111111111, 'Bingöl': 79200.0, 'Bitlis': 26800.0, 'Bolu': 56273.4375, 'Burdur': 39130.5, 'Bursa': 77510.22648083624, 'Denizli': 66540.31847133758, 'Diyarbakır': 111486.95652173914, 'Düzce': 52372.846153846156, 'Edirne': 55229.166666666664, 'Elazığ': 54994.80487804878, 'Erzincan': 59367.64705882353, 'Erzurum': 69851.58115155487, 'Eskişehir': 91988.81176470588, 'Gaziantep': 110211.19672131147, 'Giresun': 91271.66666666667, 'Gümüşhane': 56833.333333333336, 'Hakkari': 106500.0, 'Hatay': 66825.24752475247, 'Isparta': 65961.84210526316, 'Iğdır': 44785.0, 'Kahramanmaraş': 70820.95161290323, 'Karabük':

### 6. Find the average price of each brand and return it as a dictionary.


In [7]:
# Group the data by the 'City' column and compute the average price for each group
price_avg_by_city = df.groupby('City')['Price'].mean()

# Convert the Series to a dictionary
price_avg_by_city_dict = price_avg_by_city.to_dict()

print(price_avg_by_city_dict)


{'Adana': 68493.9263803681, 'Adıyaman': 72804.76190476191, 'Afyonkarahisar': 61468.0, 'Aksaray': 59964.25, 'Amasya': 49477.0, 'Ankara': 87488.58665885111, 'Antalya': 109238.6705882353, 'Ardahan': 47875.0, 'Artvin': 34460.0, 'Aydın': 59367.434782608696, 'Ağrı': 39166.5, 'Balıkesir': 54883.39361702128, 'Bartın': 74183.33333333333, 'Batman': 72102.94117647059, 'Bayburt': 65821.25, 'Bilecik': 73911.11111111111, 'Bingöl': 79200.0, 'Bitlis': 26800.0, 'Bolu': 56273.4375, 'Burdur': 39130.5, 'Bursa': 77510.22648083624, 'Denizli': 66540.31847133758, 'Diyarbakır': 111486.95652173914, 'Düzce': 52372.846153846156, 'Edirne': 55229.166666666664, 'Elazığ': 54994.80487804878, 'Erzincan': 59367.64705882353, 'Erzurum': 69851.58115155487, 'Eskişehir': 91988.81176470588, 'Gaziantep': 110211.19672131147, 'Giresun': 91271.66666666667, 'Gümüşhane': 56833.333333333336, 'Hakkari': 106500.0, 'Hatay': 66825.24752475247, 'Isparta': 65961.84210526316, 'Iğdır': 44785.0, 'Kahramanmaraş': 70820.95161290323, 'Karabük':

### 7. Find the average price of the Mercedes-Benz cars that has automatic transmission in İstanbul.


In [8]:
# Select the rows that have 'Mercedes-Benz' in the 'Brand' column and 'Automatic' in the 'Transmission' column
selected_rows = df.loc[(df['Brand'] == 'Mercedes - Benz') & (df['Tranmission'] == 'Otomatik')]

# Compute the average price of the selected rows
avg_price = selected_rows['Price'].mean()

print(avg_price)


221824.19675925927


### 8. Find the number of cars whose price is less than 100K TL, between 100K(exclusive) and 200k (inclusive) and larger than 200k(exclusive). The output should be dictionary.


In [9]:
# Create a new column with a categorical value based on the price
df['Price Range'] = df['Price'].apply(
    lambda x: '0-100' if x < 100000 else ('100 - 200' if x <= 200000 else '200 and over'))

# Count the number of cars in each category
counts = df['Price Range'].value_counts()

# Convert the Series to a dictionary
counts_dict = counts.to_dict()

print(counts_dict)


{'0-100': 6181, '100 - 200': 1237, '200 and over': 890}


### 9. Write a function which generates the lists above automatically.

In [10]:
def count_cars_by_price_range(df, ranges):
    # Initialize a dictionary to store the counts for each price range
    counts_dict = {f"0-{ranges[0]}": 0}
    for i in range(len(ranges)-1):
        counts_dict[f"{ranges[i]}-{ranges[i+1]}"] = 0
    counts_dict[f"{ranges[-1]} and over"] = 0

    # Iterate over the prices and increment the appropriate count
    for price in df['Price']:
        if price < ranges[0] * 1000:
            counts_dict[f"0-{ranges[0]}"] += 1
        elif price >= ranges[-1] * 1000:
            counts_dict[f"{ranges[-1]} and over"] += 1
        else:
            for i in range(len(ranges)-1):
                if ranges[i] * 1000 <= price < ranges[i+1] * 1000:
                    counts_dict[f"{ranges[i]}-{ranges[i+1]}"] += 1
                    break

    return counts_dict


In [11]:
# Test the function
counts_dict = count_cars_by_price_range(df, [50, 100, 150, 200, 250])
print(counts_dict)


{'0-50': 3799, '50-100': 2382, '100-150': 784, '150-200': 448, '200-250': 158, '250 and over': 737}


### 10. Find the number of cars that has an automatic transmission whose price is less than 100K TL, between 100K(exclusive) and 200k (inclusive) and larger than 200k(exclusive).


In [12]:
# Count the number of cars in each price range
counts_dict = count_cars_by_price_range(df, [100, 200])

# Find the number of cars whose price is less than 100K TL
less_than_100K = counts_dict["0-100"]

# Find the number of cars whose price is between 100K(exclusive) and 200k (inclusive)
between_100K_and_200K = counts_dict["100-200"]

# Find the number of cars whose price is larger than 200k(exclusive)
over_200K = counts_dict["200 and over"]

# Print the counts
print(f"Less than 100: {less_than_100K}")
print(f"Between 100 and 200: {between_100K_and_200K}")
print(f"Larger than 200: {over_200K}")


Less than 100: 6181
Between 100 and 200: 1232
Larger than 200: 895


### 11. For every brand, find the difference for the average prices of automatic transmission and manual transmission and return it as a dictionary.


In [13]:
# Group the DataFrame by the 'Brand' column and the 'Transmission' column
df_grouped = df.groupby(['Brand', 'Tranmission'])

# Calculate the average price for each group
df_mean = df_grouped['Price'].mean()

# Convert the resulting Series to a DataFrame
df_mean = df_mean.reset_index()

# Pivot the DataFrame to create columns for each brand and transmission type
df_pivot = df_mean.pivot(index='Brand', columns='Tranmission', values='Price')

# Calculate the difference between the average prices for automatic and manual transmission for each brand
df_diff = df_pivot['Otomatik'].subtract(df_pivot['Manuel'])

# Convert the resulting Series to a dictionary
diff_dict = df_diff.to_dict()

print(diff_dict)


{'Alfa Romeo': 61746.05263157895, 'Audi': 193193.29770444764, 'BMW': 126054.5900297619, 'Bentley': nan, 'Chevrolet': 116794.31682496608, 'Chrysler': 45766.24396135265, 'Citroën': 28825.0, 'Dacia': nan, 'Ferrari': 320501.5, 'Fiat': 16245.912903225806, 'Ford': 14589.256944444445, 'Honda': 4754.658455181176, 'Hyundai': 18558.901361796103, 'Jaguar': 102530.26666666666, 'Kia': 26238.46752893125, 'Lada': nan, 'Lancia': 145062.5, 'Maserati': 399772.1875, 'Mazda': 40613.57757885763, 'Mercedes - Benz': 187574.9813746439, 'Mini': 12906.159090909096, 'Nissan': 179088.06844597505, 'Opel': 21115.19720382634, 'Peugeot': 15391.659648361194, 'Porsche': 540567.8792517006, 'Renault': 10776.616571428574, 'Rover': 14640.5, 'Seat': 24065.064671814667, 'Skoda': 45389.46064942212, 'Smart': nan, 'Subaru': -21222.475, 'Suzuki': 9210.193779904308, 'Tata': nan, 'Tofaş': nan, 'Toyota': -5249.250398724078, 'Volkswagen': 17530.88711909709, 'Volvo': 13544.894697812386}


### 12. Create a new dataframe which is ordered in terms of the prices of the cars.


In [14]:
pd_sorted_by_price = df.sort_values('Price', ascending=False)

# Convert the DataFrame to a excel file
pd_sorted_by_price.to_excel('cars_sorted_by_price.xlsx', index=False)


### 13. Create a new dataframe which is ordered in terms of the cities first and then prices of the cars.


In [ ]:
pd_sorted_by_city_and_price = df.sort_values(['City', 'Price'], ascending=[True, False])

# Convert the DataFrame to a excel file
pd_sorted_by_city_and_price.to_excel('cars_sorted_by_city_and_price.xlsx', index=False)


### 14. Divide the price of the cars by its power (Cost/power) and add it as a new column to the datafame.


In [ ]:
# Create a new column with the price per horsepower and add it to the DataFrame
df['CostPerPower'] = df['Price'] / df['Power']

# Convert the DataFrame to a excel file
df.to_excel('cars_with_cost_per_power.xlsx', index=False)


### 15. How many different series each brand have? The output should be a dictionary.


In [ ]:
# Group the DataFrame by the 'Brand' column and count how many series each brand has
df_grouped = df.groupby('Brand')['Series'].nunique()

# count how many series each brand has and create a dictionary
brand_series_count_dict = {}

for brand, count in df_grouped.items():
    brand_series_count_dict[brand] = count

print(brand_series_count_dict)


### 16. Write a function called analyze_car_data.


In [ ]:
def analyze_car_data(count_type, property_type):
    if count_type == "Price":
        return df.groupby(property_type)["Price"].mean().to_dict()
    elif count_type == "Count":
        return df[property_type].value_counts().to_dict()
    elif count_type == "Power":
        return df.groupby(property_type)["Power"].mean().to_dict()
    else:
        raise ValueError("Invalid count type. Choose from 'Price', 'Count', or 'Power'.")


In [ ]:
# If you call the function as follows then it will return the average car price of each city.
analyze_car_data("Price", "City")


In [ ]:
# For example, if you call the function as follows then it will return the number of cars for each transmission type.
analyze_car_data("Count", "Tranmission")


In [ ]:
# If you call the function as follows then it will return the average power of cars for each Brand.
analyze_car_data("Power", "Brand")
